# はじめに

scoreを求めるよりは、スコアを上げる為の情報をどう得るかを書き溜めます。<br>
学習済みmodelのsave、train結果とtargetとの差の可視化、どのfeatureが重要かの可視化を目指します。<br>
後で見返せるように、自分用にまとめています。<br>

# NOTATION

### <font color="red">I try to find hints of feature engineering by comparing model result and train data.<br>You can find visualization result of model output and train data. </font>

#### <font color="red">This notebook use the trained model of following link due to run visualize with model result quckly and easy to change model of yours.</font>
[copied model to this notebook](https://www.kaggle.com/tfukuda675/update-simple-lstm-simple-data-model)

#### <font color="red">When you use trained model in this notebook, please set "use_trained_model = True" in cell of </font>
[Settings of this notebook](https://www.kaggle.com/tfukuda675/update-simple-lstm-simple-data#Settings) <br>
note : above model is copied of this notebook result.

<font color="red">If you want to look over data visualize result, please refer save version 37</font><br>
<font color="red"><s>I create u_out = 1 data for my understanding until version 37.</s></font><br>
<font color="red">From version 38, <s>I treat u_out = 0" data!!</s></font><br>
<font color="red">From version 38, at first, create model with simple data with above stragety.</font><br>
<font color="red">Next, compare model result of train data and pressure of train data due to consider feature engineering of data.</font><br>

## New Notebook

This notebook takes long runtime to compair model result vs train data, because this notebook run all steps, create data, create model, compair and visualize difference of model result and train data.<br>
So, I am editing new notebooks as following, due to run all steps quickly and easy to swap of your data or models.<br>

* https://www.kaggle.com/tfukuda675/simple-lstm-create-train-data
* https://www.kaggle.com/tfukuda675/simple-lstm-run-train
* https://www.kaggle.com/tfukuda675/simple-lstm-compair-model-result

## train data strategy (戦略)
timestepに直線性が無いデータがあります。<br>
pressure(ターゲットですが)に負のものがあります。<br>
評価に関係しないu_out = 0のデータもあります。<br>
そういったキレイではないデータを削除して学習します。<br>
testデータにはキレイでないデータも入っていますが、無視する戦略です。<br>

Strategy of this notebook is "simplify data as much as possible"
* Ignore non liner time_step data
* Ignore negative pressure data
* <s>use u_out = 0 data only for train/test</s><br>
Please let me now, when you have any recommendations.<br>

At first, create model with above stragety.<br>
Next, compare model result with train data and train pressure data.<br>
Then consider additional feature of data with above result.<br>

## refer to for visualize
過去に作ったものです。plotly/seaborn/matplotlibが期待するdfの形状に言及しています。<br>
https://www.kaggle.com/tfukuda675/data-visualization-plotly-seaborn-matplot?scriptVersionId=76838377&cellId=1



# Visualize original data u_in histgram for each time step id

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize, StandardScaler
from sklearn.model_selection import train_test_split, GroupKFold, KFold


path = '../input/ventilator-pressure-prediction'
train = pd.read_csv(f"{path}/train.csv")
train["time_step_id"] = list(range(1,81,1)) * int(len(train)/80)
range_bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65,70,75,80,85,90,95,100]
bins_name = ['~5', '~10', '~15','~20','~25', '~30', '~35', '~40','~45', '~50', '~55', '~60','~65','~70','~75','~80','~85','~90','~95','~100']
train["u_in_range"] = pd.cut(train["u_in"],bins=range_bins,labels=bins_name)
tmp_df = pd.DataFrame()
for k,grp in train.groupby("time_step_id"):
    tmp_df = tmp_df.append(grp["u_in_range"].value_counts())
tmp_df = tmp_df.reset_index(drop=True)
tmp_df.columns = tmp_df.columns.astype(str)
tmp_df = tmp_df.reindex(columns=bins_name)
ax = sns.heatmap(tmp_df)
ax.set_xlabel("u_in_range", fontsize = 20)
ax.set_ylabel("time_step_id", fontsize = 20)
plt.show()

# Visualize u_in hist with StandardScaler() for each time step id

In [ ]:
train = pd.read_csv(f"{path}/train.csv")
columns = train.columns
#scaler = RobustScaler()
scaler = StandardScaler()
train = scaler.fit_transform(train)
train = pd.DataFrame(train,columns= columns)
train["time_step_id"] = list(range(1,81,1)) * int(len(train)/80)
range_bins = [-1.8, -1.6, -1.4, -1.2, -1.0, -0.8, -0.6, -0.4,-0.2,0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8]
bins_name = ['~-1.6', '~-1.4', '~-1.2','~-1.0','~-0.8','~-0.6','~-0.4','~-0.2','~0','~0.2','~0.4','~0.6','~0.8','~1.0','~1.2','~1.4','~1.6','~1.8']
train["pressure_range"] = pd.cut(train["pressure"],bins=range_bins,labels=bins_name)
tmp_df = pd.DataFrame()
for k,grp in train.groupby("time_step_id"):
    tmp_df = tmp_df.append(grp["pressure_range"].value_counts())
tmp_df = tmp_df.reset_index(drop=True)
tmp_df.columns = tmp_df.columns.astype(str)
tmp_df = tmp_df.reindex(columns=bins_name)
ax = sns.heatmap(tmp_df)
ax.set_xlabel("pressure_range", fontsize = 20)
ax.set_ylabel("time_step_id", fontsize = 20)
plt.show()

# Visualize u_in hist with RobustScaler() for each time step id

In [ ]:
train = pd.read_csv(f"{path}/train.csv")
columns = train.columns
scaler = RobustScaler()
#scaler = StandardScaler()
train = scaler.fit_transform(train)
train = pd.DataFrame(train,columns= columns)
train["time_step_id"] = list(range(1,81,1)) * int(len(train)/80)
#range_bins = [-0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
range_bins = [-9, -8, -7, -6, -5, -4, -3, -2,-1,0,1,2,3,4,5,6,7,8,9,10]
bins_name = ['~-8', '~-7','~-6','~-5','~-4','~-3','~-2','~-1','~0','~1','~2','~3','~4','~5','~6','~7','~8','~9','10']
#train["pressure_range"] = pd.cut(train["pressure"],bins=range_bins,labels=bins_name)
train["u_in_range"] = pd.cut(train["u_in"],bins=range_bins,labels=bins_name)
tmp_df = pd.DataFrame()
for k,grp in train.groupby("time_step_id"):
    #tmp_df = tmp_df.append(grp["pressure_range"].value_counts())
    tmp_df = tmp_df.append(grp["u_in_range"].value_counts())
tmp_df = tmp_df.reset_index(drop=True)
tmp_df.columns = tmp_df.columns.astype(str)
tmp_df = tmp_df.reindex(columns=bins_name)
ax = sns.heatmap(tmp_df)
ax.set_xlabel("u_in_range", fontsize = 20)
ax.set_ylabel("time_step_id", fontsize = 20)
plt.show()

# Codes that surprised me recently

## use for in groupby()

It can modify data after groupby.
groupbyしたあとのdfを使いたい事が多々あったのですが、まさかforで実現できるとは思いませんでした。

```
for key, grp in train.groupby('breath_id'):
    fig.add_trace(go.Violin(x=grp["u_in"]))
```
<br><br>

## use tqdm in groupby()

Can use tqdm in groupby process.

```
from tqdm import tqdm
tqdm.pandas()
for key, grp in tqdm(train.groupby('breath_id')):
    fig.add_trace(go.Violin(x=grp["u_in"]))
```


## 任意の範囲でheatmap

出現頻度を任意の範囲で確認したいことがあったのですが、binsを作って、それをcut()で利用することで実現可能でした。

```
range_bins = pd.interval_range(start=-10, freq=10, end=100)
pd.cut(grp,bins=range_bins).value_counts()
```
<br><br>

# 用語集

### twinx @ matplotlib

matplotlibで左右のx軸を使う宣言。

https://qiita.com/supersaiakujin/items/e2ee4019adefce08e381
<br><br>

### RobustScaler @scikit-learn

scikit-learn を使った特徴量のスケーリング方法の一つ。<br>
第1分位点 Q_1Q と第3分位点 Q_3Q の範囲が1になるように変換。<br>
外れ値に引っ張られるのを防ぐ。<br>

https://pystyle.info/ml-feature-scaling/

このノートブックでは使わない。<br>

<br><br>

### fitとtransformについて
fitで最大・最小を計算<br>
mmscaler.fit(x)           # xの最大・最小を計算<br>
transformで値をfitの間に入るように変換<br>
y = mmscaler.transform(x) # xを変換<br>

trainでfitで最大・最小を計算<br>
transformをtrainとtestへ実施。<br>

【注意】<br>
RobustScalerのfitで値を変換するその内容を決定し、transformで値の変更を実施する。<br>
そのため、fitしたときのcolumnの幅と、transformするときのcolumnの幅が揃わないとだめ。<br>

<br><br>
### add_prefix / add_suffix @ dataframe

列名にプレフィックス（接頭辞）、サフィックス（接尾辞）を追加するためのメソッド。
```
print(df.add_prefix('X_'))
#        X_A  X_B  X_C
```
https://note.nkmk.me/python-pandas-dataframe-rename/
<br><br>

### 行列の@ @ numpy / tf

dotの意味。numpyとtfで使える。

```
A = np.random.rand(3,3)
B = tf.random.uniform((3,3))

AAA = A @ A @ A # AAA = A.dot(A).dot(A)
BBB = B @ B @ B # BBB = tf.matmul(ft.matmul(B,B),B)
```
<br><br>

### .agg(list) @ groupby()
groupbyした項目以外のデータをまるまるlistにする。

```
df2 = df.groupby('breath_id').agg(list).reset_index()
```

|breath_id|id|R|C|time_step|u_in|u_out|pressure|
|---|---|---|---|---|---|---|---|
|0|1|[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...|[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...|[50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 5...|[0.0, 0.0336523056030273, 0.067514419555664, 0...|[0.0833340056346443, 18.38304147263472, 22.509...|[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...|[5.837491705069121, 5.907793850520346, 7.87625...|


<br><br>
### tf.test.is_gpu_available()

GPUが使えるかどうか。使えればTrueが帰ってくる。

<br><br>

### model.save @ tf
mode.save()でディレクトリを指定。.hd5をつければhd5で保存。<br>

利用は次の通り。.hd5であれば拡張子も忘れずに。<br>
```
new_model = tf.keras.models.load_model('saved_model/my_model')
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
```
https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ja#savedmodel_%E5%BD%A2%E5%BC%8F

<br><br>
### model.save_weights @ tf

学習中や終了中にsaveが可能。weightだけ保存するので、同じmodel構成を作れば他のtfインスタンスにコピーが可能。<br>
ちょっとめんどくさそうだけど、こっちのほうが良さそう。

https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja#%E8%A8%93%E7%B7%B4%E4%B8%AD%E3%81%AB%E3%83%81%E3%82%A7%E3%83%83%E3%82%AF%E3%83%9D%E3%82%A4%E3%83%B3%E3%83%88%E3%82%92%E4%BF%9D%E5%AD%98%E3%81%99%E3%82%8B

```
# ファイル名に(`str.format`を使って)エポック数を埋め込む
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 5エポックごとにモデルの重みを保存するコールバックを作成
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 新しいモデルのインスタンスを作成
model = create_model()

# `checkpoint_path` フォーマットで重みを保存
model.save_weights(checkpoint_path.format(epoch=0))

# 新しいコールバックを使い、モデルを訓練
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)
```

以下でモデルをチェック
ls {checkpoint_dir}

```
checkpoint            cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index        cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index        cp-0035.ckpt.index
```
最新版の利用は
```
latest = tf.train.latest_checkpoint(checkpoint_dir)

# 新しいモデルのインスタンスを作成
model = create_model()

# 先ほど保存した重みを読み込み
model.load_weights(latest)

# モデルを再評価
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
```
<br><br>

### Dense layer　全結合層

次のコードの結果が下の絵。
```
model = models.Sequential()
model.add(Dense(10, activation='relu', input_shape=(4,)))   
model.add(Dense(3, activation='softmax'))
```

<img src="https://sinyblog.com/wp/wp-content/uploads/2019/04/keras-003.jpg" width="500">

<br><br>

### selu @ 活性化関数 tf dense layer

勾配喪失に強い活性化関数。Reluの時代は終わった、SeLuらしい。

https://data-analysis-stats.jp/%E6%B7%B1%E5%B1%9E%E5%AD%A6%E7%BF%92/%E6%B4%BB%E6%80%A7%E5%8C%96%E9%96%A2%E6%95%B0%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81/

<img src="https://data-analysis-stats.jp/wp-content/uploads/2021/03/SELU-activation-01.png" width="500">

<br><br>

### keras.layers.Bidirectional

従来のRNNのニューロンをフォワード(未来)なものとバックワード(過去)なものとの2つに分ける。<br>
今回は必要かな？肺モデルからのフィードバックが入力側にあるなら、Bidirectionalが必要そうだけど。<br>

https://www.i2tutorials.com/deep-dive-into-bidirectional-lstm/

<img src="https://d1zx6djv3kb1v7.cloudfront.net/wp-content/media/2019/05/Deep-Dive-into-Bidirectional-LSTM-i2tutorials.jpg" width="500">



# ライブラリ読み込み

In [ ]:
#!pip install joypy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
#from joypy import joyplot for matplotlib
import tensorflow as tf
from tqdm import tqdm
import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize, StandardScaler
from sklearn.model_selection import train_test_split, GroupKFold, KFold

tqdm.pandas()

%matplotlib inline

In [ ]:
if tf.device('/CPU:0'):
    print(tf.device('/CPU:0'))

# Settings

In [ ]:
# When Debug mode is true, this notebook use small train data.
debug_mode = True
debug_data_count = int(1000)

tqdm.pandas()

# folloiwng settig takes time due to plotly is so slow.
visualize = False  # [True | False]
visualize2 = False

# Use trained model. no run training. Just use trained model only.

use_trained_model = True

# Read data

In [ ]:
path = '../input/ventilator-pressure-prediction'
train = pd.read_csv(f"{path}/train.csv")
test = pd.read_csv(f"{path}/test.csv")
submission = pd.read_csv(f'{path}/sample_submission.csv')

# confirm linearity of time_step

In [ ]:
if visualize:
    time_step_diff_limit = 0.04
    non_liner_timestep_breath_ids = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        diff_se = grp["time_step"].diff()
        diff_chk = diff_se[diff_se > time_step_diff_limit]
        if len(diff_chk) != 0:
            non_liner_timestep_breath_ids.append(k)
#
#print(non_liner_timestep_breath_ids)
## results are following:
## [803, 2327, 3178, 4199, 5830, 10277, 11502, 13238, 15803, 16315, 16634, 18117, 18600, 24127, 25397, 28189, 28942, 30181, 32296, 36128, 36175, 37711, 38237, 38415, 39045, 39722, 42317, 42988, 43344, 44245, 45197, 46324, 49849, 53877, 54129, 55244, 55851, 61454, 64662, 67422, 67748, 72104, 74766, 76037, 78768, 79105, 80375, 87127, 87776, 89084, 91883, 93186, 98677, 102063, 104001, 106034, 107067, 109693, 111439, 112027, 115588, 119689, 120878, 121135, 125136]

# visualize non linearity time_step (直線でないtime_stepの可視化)

In [ ]:
if visualize:
    non_liner_timestep_df = train[train["breath_id"].isin(non_liner_timestep_breath_ids)]
    fig = go.Figure()
    for k,grp in non_liner_timestep_df.groupby("breath_id"):
        grp = grp.reset_index(drop=True)
        fig.add_trace(go.Scatter(x=grp.index, y=grp["time_step"], mode='lines', name=k))
    fig.show()

# visualize linearity time_step (直線なtime_stepの可視化)

おっと、傾きが色々とある！
計測機器側の計測インターバルがずれるとは思えないので、一回のサイクルを80分割したのだと思う。

In [ ]:
if visualize:
    liner_timestep_df = train[~train["breath_id"].isin(non_liner_timestep_breath_ids)]
    fig = go.Figure()
    for k,grp in tqdm(liner_timestep_df[:80*10000].groupby("breath_id")):
        grp = grp.reset_index(drop=True)
        fig.add_trace(go.Scatter(x=grp.index, y=grp["time_step"], mode='lines', name=k))
    fig.show()

# find minus pressure data (ターゲット値ですが)にマイナス値を持つデータを探す

In [ ]:
if visualize:
    minus_pressure_breath_ids = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        m = grp["pressure"].min()
        if m < 0:
            minus_pressure_breath_ids.append(k)
# print(minus_pressure_breath_ids)
# [542, 851, 3928, 7949, 11216, 13594, 16599, 19236, 20075, 22164, 23710, 27195, 27731, 30127, 36474, 40431, 40753, 43103, 43630, 44309, 45099, 45681, 45877, 46018, 46020, 46486, 47325, 49376, 49941, 50459, 52137, 53057, 54206, 56152, 56760, 57119, 58835, 59101, 60949, 65596, 67080, 67788, 70753, 71461, 72011, 74977, 77803, 83713, 85391, 86508, 90584, 91132, 91464, 92955, 94037, 97520, 98041, 98080, 101951, 106703, 108406, 109424, 109761, 110499, 111419, 112036, 113323, 113639, 118131, 119582, 120445, 121570, 124575]

# visualize minus pressure data (負のpressureを持つデータを可視化)

In [ ]:
if visualize:
    minus_pressure_df = train[train["breath_id"].isin(minus_pressure_breath_ids)]
    minus_pressure_df_plotly = pd.melt(minus_pressure_df,id_vars=["time_step","breath_id"], value_vars=["pressure"])
    fig = px.line(minus_pressure_df_plotly, x="time_step" , y="value",color = "variable",line_group ="breath_id")
    for line in fig.data:
        line['line']['color']='rgba(0, 0, 255, 0.1)'
    fig.show()

# count steps of u_out = 1   (排気段階)のstep数をtrain/testデータで確認

In [ ]:
if visualize:
    u_out_open_step_counts = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[1]
        u_out_open_step_counts.append(count)
    
    u_out_open_step_counts_test = list()
    for k, grp in tqdm(test.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[1]
        u_out_open_step_counts_test.append(count)

# visualize by histgram counts of u_out = 1 (排気段階)のstep数の分布を可視化
48 ~ 51にまとまっている。裾のは、55まで。

In [ ]:
if visualize: 
    fig = px.histogram(x=u_out_open_step_counts,nbins=8)
    fig.update_layout(title="u_out = 1 count histgram in train")
    fig.show()

In [ ]:
if visualize:
    fig = px.histogram(x=u_out_open_step_counts_test,nbins=8)
    fig.update_layout(title="u_out = 1 count histgram in test")
    fig.show()

# data count of counts over 52 of u_out = 1 (排気段階)のstep数が52以上のデータ数

In [ ]:
if visualize: 
    u_out_open_step_counts_over52 = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[1]
        if count > 51:
            u_out_open_step_counts_over52.append(count)
    len(u_out_open_step_counts_over52)

# visualize pressuer of u_out = 1

In [ ]:
if visualize:
    train = train[:80*1000]
    train = train[train["u_out"] == 1]
    pressure_df_uout = pd.melt(train,id_vars=["time_step","breath_id"], value_vars=["pressure"])
    fig = px.line(pressure_df_uout, x="time_step" , y="value",color = "variable",line_group ="breath_id")
    for line in fig.data:
        line['line']['color']='rgba(0, 0, 255, 0.05)'
    fig.show()
    

# count steps of u_out = 0 step

In [ ]:
if visualize2:
    u_out_close_step_counts = list()
    for k, grp in tqdm(train.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[0]
        u_out_close_step_counts.append(count)
    
    u_out_close_step_counts_test = list()
    for k, grp in tqdm(test.groupby("breath_id")):
        count = grp.groupby("u_out")["id"].count()[0]
        u_out_close_step_counts_test.append(count)

In [ ]:
if visualize2: 
    fig = px.histogram(x=u_out_close_step_counts,nbins=8)
    fig.update_layout(title="u_out = 0 count histgram in train")
    fig.show()

In [ ]:
if visualize2:
    fig = px.histogram(x=u_out_close_step_counts_test,nbins=8)
    fig.update_layout(title="u_out = 0 count histgram in test")
    fig.show()

# Utilitys

data_cleanの中で、直線性の無いデータ、負のpressureを持つデータ、u_out = 1のstep数52以上あるデータを削除。<br>

In [ ]:
def data_clean(df):
    ## pickup ignore breath id
    ignore_breath_ids = set()
    
    
    time_step_diff_limit = 0.04
    for k, grp in tqdm(df.groupby("breath_id")):
        
        ## ignore non liner time_step data
        diff_se = grp["time_step"].diff()
        diff_chk = diff_se[diff_se > time_step_diff_limit]
        if len(diff_chk) != 0:
            ignore_breath_ids.add(k)
            
        ## ignor negative pressure data
        m = grp["pressure"].min()
        if m < 0:
            ignore_breath_ids.add(k)
            
        ## ignore (u_out = 0 step) < 29
        #count0 = grp.groupby("u_out")["id"].count()[0]
        #if count0 < 29:
        #    ignore_breath_ids.add(k)
    
    df = df[~df["breath_id"].isin(np.array(list(ignore_breath_ids)))]
    return df

def change_type(df):
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    return df

def add_features(df):
    df['u_in_cumsum'] = df.groupby('breath_id')['u_in'].cumsum()
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df = df.fillna(0)
    return df


def tf_tpu_or_gpu_or_cpu():
    tpu = None
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        return "tpu"

    elif tf.test.is_gpu_available():
        strategy = tf.distribute.get_strategy()
        print('Running on GPU')
        return "gpu"

    else:
        strategy = tf.distribute.get_strategy()
        print('Running on CPU')
        return "cpu"


# 排気時のみのデータ作成

u_outのstep数最大(今回は51)でcut。短い場合は、一番最後のデータをコピーして51に合わせる。

In [ ]:
def exhaust_mode_df(df):
    grp_len = int(51)
    new_df = pd.DataFrame()
    for k, grp in df.groupby("breath_id"):
        tmp_df = grp[grp["u_out"] == 1]
        rowno = tmp_df.shape[0]
        for l in range(grp_len - rowno):
            tmp_df = tmp_df.append(grp.tail(1),ignore_index=True)
        new_df = new_df.append(tmp_df,ignore_index=True)
    return new_df

In [ ]:
def uout0_mode_df(df):
    grp_len = int(32)
    new_df = pd.DataFrame()
    for k, grp in df.groupby("breath_id"):
        tmp_df = grp[grp["u_out"] == 0]
        rowno = tmp_df.shape[0]
        if "pressure" in tmp_df.columns:
            for l in range(grp_len - rowno):
                tmp_df = tmp_df.append({""},ignore_index=True)
        else:
            for l in range(grp_len - rowno):
                tmp_df = tmp_df.append(grp.tail(1),ignore_index=True)
        new_df = new_df.append(tmp_df,ignore_index=True)
    return new_df

# データ読み込みとconfig設定

In [ ]:
path = '../input/ventilator-pressure-prediction'
train = pd.read_csv(f"{path}/train.csv")
test = pd.read_csv(f"{path}/test.csv")
submission = pd.read_csv(f'{path}/sample_submission.csv')

# Debug モード時の処理
データ数を削減<br>
In use_trained_model case, use new data.

In [ ]:
if debug_mode:
    if use_trained_model:
        train = train[80*debug_data_count:80*debug_data_count*2]
        test = test[80*debug_data_count:80*debug_data_count*2]
    else:
        train = train[:80*debug_data_count]
        test = test[:80*debug_data_count]

# apply utilitys for data
データの追加と削除がメイン

In [ ]:
# exhaust_mode_df() do not work collectry
u_out1_only = False


## trainのみ、data_cleanを実施
train = data_clean(train)  ## time_stepがリニア出ないものは削除
if u_out1_only:
    train = exhaust_mode_df(train)
train = add_features(train)
train = change_type(train)

if u_out1_only:
    test = exhaust_mode_df(test)
test = add_features(test)
test = change_type(test)

# choose useful data for train and test and create target data
targetデータの作成と、不要なcolumnの削除。

In [ ]:
if u_out1_only:
    targets = train[['pressure']].to_numpy().reshape(-1, 51)
    train = train.drop(['pressure', 'id', 'breath_id','u_out'], axis=1)
    test = test.drop(['id', 'breath_id','u_out'], axis=1)
else:
    targets = train[['pressure']].to_numpy().reshape(-1, 80)
    train = train.drop(['pressure', 'id', 'breath_id'], axis=1)
    test = test.drop(['id', 'breath_id'], axis=1)


# 入力データのデータそのものの変形

### RobustScalerについて
入力データの分布に応じで変形。外れ値に強くしたり、正規分布に近くしたりできる。<br>
https://helve-blog.com/posts/python/scikit-learn-feature-scaling/
<br><br>
### numpy bload cast error について
原因は、trainとtestのデータの幅が違ったから。<br>
RobustScalerのfitで値を変換するその内容を決定し、transformで値の変更を実施する。<br>
そのため、fitしたときのcolumnの幅と、transformするときのcolumnの幅が揃わないとだめ。<br>
https://www.headboost.jp/numpy-array-broadcasting/
<br><br>

In [ ]:
#scaler = RobustScaler()
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# データ形状をTF向けに変更
breath_idごとのデータになるように整形。<br>

In [ ]:
if u_out1_only:
    train = train.reshape(-1, 51, train.shape[-1])
    test = test.reshape(-1, 51, test.shape[-1])
else:
    train = train.reshape(-1, 80, train.shape[-1])
    test = test.reshape(-1, 80, test.shape[-1])

# run tf function

In [ ]:
def run_tf_blstm(epoch=int(50),batch_size=int(1024),train=None,test=None,targets=None):
    #kf = KFold(n_splits=5, shuffle=True, random_state=2000)
    kf = KFold(n_splits=5, shuffle=True)
    test_preds = []
    test_history = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        model = keras.models.Sequential([
            #keras.layers.Embedding(input_dim=train.shape[-2:], output_dim=300, mask_zero=True),
            keras.layers.Input(shape=train.shape[-2:]),
            keras.layers.Bidirectional(keras.layers.LSTM(300, return_sequences=True)),          
            keras.layers.Bidirectional(keras.layers.LSTM(250, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)),
            keras.layers.Dense(50, activation='selu'),
            keras.layers.Dense(1),
        ])
        model.compile(optimizer="adam", loss="mae")    
        
        scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/batch_size), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=1)

        #es = EarlyStopping(monitor="val_loss", patience=15, verbose=1, mode="min", restore_best_weights=True)

        history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epoch, batch_size=batch_size, callbacks=[lr])
        test_history.append(history.history)
        model.save(f'model_save_fold{fold+1}')
        #test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())
        test_preds.append(model.predict(test).squeeze())
    
    return test_preds, test_history

def run_tf_lstm(epoch=int(50),batch_size=int(1024),train=None,test=None,targets=None):
    #kf = KFold(n_splits=5, shuffle=True, random_state=2000)
    kf = KFold(n_splits=5, shuffle=True)
    test_preds = []
    test_history = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        model = keras.models.Sequential([
            #keras.layers.Embedding(input_dim=train.shape[-2:], output_dim=300, mask_zero=True),
            keras.layers.Input(shape=train.shape[-2:]),
            keras.layers.LSTM(300, return_sequences=True),
            keras.layers.LSTM(250, return_sequences=True),
            keras.layers.LSTM(150, return_sequences=True),
            keras.layers.LSTM(100, return_sequences=True),
            keras.layers.Dense(50, activation='selu'),
            keras.layers.Dense(1),
        ])
        model.compile(optimizer="adam", loss="mae")    
        
        scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/batch_size), 1e-5)
        lr = LearningRateScheduler(scheduler, verbose=1)

        #es = EarlyStopping(monitor="val_loss", patience=15, verbose=1, mode="min", restore_best_weights=True)

        history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epoch, batch_size=batch_size, callbacks=[lr])
        test_history.append(history.history)
        model.save(f'model_save_fold{fold+1}')
        result = model.predict(test)
        #print(result.shape) #(50300, 80, 1)
        #print(result.squeeze().shape)  #(50300, 80)
        #print(result.squeeze().reshape(-1,1).shape) #(4024000, 1)
        #test_preds.append(result.squeeze().reshape(-1, 1).squeeze())
        test_preds.append(result.squeeze())
    
    return test_preds, test_history

# Train と Test

## 戦略
* full randamでロバスト性を狙う。外れ値は外したデータなので、randamで振られても安定するはず。
* modelをsave。後で答えとどれだけずれているか見る。
* <s>use only u_out = 1 data.</s>

A.I<br>
check 

In [ ]:
EPOCH = 200
BATCH_SIZE = 1024
test_preds = list()

device = tf_tpu_or_gpu_or_cpu()

if use_trained_model:
    pass
else:
    if device == "cpu" :
        test_preds,history = run_tf_blstm(epoch=EPOCH,batch_size=BATCH_SIZE,train=train,test=test,targets=targets)

    elif device == "gpu":
        test_preds,history = run_tf_blstm(epoch=EPOCH,batch_size=BATCH_SIZE,train=train,test=test,targets=targets)
    elif device == "tpu":
        try:
            with tpu_strategy.scope():
                test_preds,history = run_tf_blstm(epoch=EPOCH,batch_size=BATCH_SIZE,train=train,test=test,targets=targets)
        except :
            print('Error')

# re shape prediction results

In [ ]:
if use_trained_model:
    pass
else:
    preds = np.sum(test_preds, axis=0).tolist()

# re read test.csv and add pred results to "pressure"

In "u_out1_only = False" case, do not need to run following function.<br>
Just only for "u_out1_only = True" case.

In [ ]:
def add_pressure_data(df,preds):
    index = 0
    new_df = pd.DataFrame()
    for k,grp in df.groupby("breath_id"):
        u_out_len = len(grp[grp["u_out"] == 1])
        if u_out_len < 52:
            grp.loc[grp["u_out"] == 1,"pressure"] = preds[index][:u_out_len]
        else:
            out_preds = preds[index]
            for l in range(u_out_len - 51):
                out_preds = out_preds.append(out_preds[-1])
            grp.loc[grp["u_out"] == 1,"pressure"] = out_preds
        new_df = new_df.append(grp,ignore_index=True)
        index += 1
    return new_df

# In non debug mode, create submission data

In [ ]:
if debug_mode:
    pass
else:
    submission["pressure"] = preds
    submission.to_csv('submission.csv', index=False)

# Load Pre Train model

In [ ]:
if use_trained_model:
    model = tf.keras.models.load_model('../input/update-simple-lstm-simple-data-model/update-simple-lstm-simple-data/model_save_fold5')
    result = model.predict(train)
    result = result.squeeze()

# Compare model result and train data, And visualize

In [ ]:
## res and tgt is numpy array
for res, tgt in zip(result,targets):
    abs_diff =  np.sum(np.abs(res - tgt))
    if abs_diff > 500:
        df = pd.DataFrame(res)
        df = pd.concat([df, pd.DataFrame(tgt)], axis=1)
        df.columns = ["res","tgt"]
        df["id"] = df.index
        df = pd.melt(df,id_vars=["id"])
        plt.figure()
        fig = sns.lineplot(data=df, x='id', y="value", hue="variable")
        fig.set_title(f"abs_diff : {abs_diff}")
    elif abs_diff < 30:
        df = pd.DataFrame(res)
        df = pd.concat([df, pd.DataFrame(tgt)], axis=1)
        df.columns = ["res","tgt"]
        df["id"] = df.index
        df = pd.melt(df,id_vars=["id"])
        plt.figure()
        fig = sns.lineplot(data=df, x='id', y="value", hue="variable")
        fig.set_title(f"abs_diff : {abs_diff}")

# 